In [12]:
import math as m
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
%matplotlib inline



In [13]:
def S(basis) :
    N = basis.shape[0]
    Overlap_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Overlap_matrix[p,q] = (np.pi/(basis[p] + basis[q]))**(3/2)
            Overlap_matrix[q,p] = Overlap_matrix[p,q]
    return Overlap_matrix

def T(basis) :
    N = basis.shape[0]
    Kintetic_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Kintetic_matrix[p,q] = 3*basis[p]*basis[q]*np.pi**(3/2)/(basis[p] + basis[q])**(5/2)
            Kintetic_matrix[q,p] = Kintetic_matrix[p,q]
    return Kintetic_matrix

def A(basis) :
    N = basis.shape[0]
    Coulomb_matrix = np.zeros((N,N))
    for p in range(N) :
        for q in range(p,N) :
            Coulomb_matrix[p,q] = -4*np.pi/(basis[p] + basis[q])
            Coulomb_matrix[q,p] = Coulomb_matrix[p,q]
    return Coulomb_matrix


In [14]:
def Q(basis) : 
    N = basis.shape[0]
    Electron_matrix = np.zeros((N,N,N,N))
    for p in range(N) :
        for q in range(p+1) :
            for r in range(p+1) :
                if p != q :
                    for s in range(r+1) :
                        Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)
                        Electron_matrix[p,s,q,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,r,p,s] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,s,p,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,p,s,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,q,s,p] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,p,r,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,q,r,p] = Electron_matrix[p,r,q,s]
                else :
                    for s in range(q+1) :
                        Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)
                        Electron_matrix[p,s,q,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,r,p,s] = Electron_matrix[p,r,q,s]
                        Electron_matrix[q,s,p,r] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,p,s,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[r,q,s,p] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,p,r,q] = Electron_matrix[p,r,q,s]
                        Electron_matrix[s,q,r,p] = Electron_matrix[p,r,q,s]
    return Electron_matrix

In [15]:
def normalize(C,S) :
    I = 0
    N = np.shape(C)[0]
    for r in range(N) :
        for s in range(N) : 
            I = I + S[r,s]*C[r]*C[s]
    C = C/I
    return C

In [22]:
def Helium(a0,a1,a2,a3,C) :
    basis = np.array([a0, a1, a2, a3])
    
    S_matrix = S(basis)
    T_matrix = T(basis)
    A_matrix = A(basis)
    Q_tensor = Q(basis)
    Q_matrix = np.zeros((4,4)) 

    C = normalize(C,S_matrix)
    
    for r in range(4) : 
        for s in range(4) : 
            Q_matrix = Q_matrix + Q_tensor[:,r,:,s]*C[r]*C[s]
    
    eigenenergy, eigenvector= eigh((T_matrix + A_matrix + Q_matrix), S_matrix)
    eigenenergy = np.min(eigenenergy)
    eigenvector = eigenvector[:,np.argmin(eigenenergy)]
    
    total_energy = 0 
    for p in range(4) :
        for q in range(4) :
            total_energy = total_energy  + 2*(A_matrix[p,q]+T_matrix[p,q])*C[p]*C[q]
            for r in range(4) :
                for s in range(4) :
                    total_energy = total_energy + Q_tensor[p,r,q,s]*C[p]*C[r]*C[q]*C[s]
                    
    return eigenenergy, eigenvector, total_energy


In [24]:
def SCF(a0,a1,a2,a3) : 
    n = 10
    C = np.array([1,1,1,1])
    for ele in range(n) :
        total_energy = Helium(a0,a1,a2,a3,C)[2]
        C = Helium(a0,a1,a2,a3,C)[1]
        print(total_energy)
    return total_energy

SCF(0.298073, 1.242567, 5.782948, 38.474970)


0.0017888309592396406
1.2394006540363702
0.9739521432611687
1.041728079523402
1.0228931540602437
1.0280135169551006
1.0266130151879143
1.0269954401124306
1.0268909667311192
1.0269195039290004


1.0269195039290004

In [18]:
def MGD(a0,a1,a2,a3,step_size,limit) :
    gradient = np.array([[SCF(a0+0.00001,a1,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1+0.00001,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2+0.00001,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2,a3+0.00001) - SCF(a0,a1,a2,a3)]])/0.00001
    v = np.zeros((4,1))
    i = 0
    data = np.array([[i],[SCF(a0,a1,a2,a3)],[a0],[a1],[a2],[a3],[np.linalg.norm(gradient)]])
    while i < limit and np.linalg.norm(gradient) > 0.000001:
        v = step_size*gradient + 0.9*v
        a0 = a0 - v[0,0]
        a1 = a1 - v[1,0]
        a2 = a2 - v[2,0]
        a3 = a3 - v[3,0]
        i += 1
        gradient = np.array([[SCF(a0+0.00001,a1,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1+0.00001,a2,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2+0.00001,a3) - SCF(a0,a1,a2,a3)], [SCF(a0,a1,a2,a3+0.00001) - SCF(a0,a1,a2,a3)]])/0.000001
        data = np.append(data,[[i],[SCF(a0,a1,a2,a3)],[a0],[a1],[a2],[a3],[np.linalg.norm(gradient)]], axis = 1)
    return data

In [19]:
test = MGD(0.247,0.891,3.49,18.52,0.1,100)
for ele in range(1,7) :
    plt.figure(ele)
    plt.plot(test[0,:],test[ele,:])
print(test[:,-1])

0.0007609663156237526
1.2413643339965972
0.9651165849175372
1.0401232460030103
1.018401144459878
1.0245840384564986
1.0228153631942205
1.023320588583937
1.0231762112737053
1.0232174649058858
0.0007608959900781492
1.2413649952536405
0.9651138672556456
1.0401223005388254
1.018399480808855
1.024582639672815
1.0228138714606292
1.0233191283108982
1.023174740606874
1.0232159976098358
0.0007609045175714564
1.2413653139003704
0.9651133852982449
1.0401221002232364
1.0183991670089918
1.0245823660765876
1.0228135839924901
1.023318845472751
1.023174456253992
1.0232157137444442
0.0007608959900781492
1.2413649952536405
0.9651138672556456
1.0401223005388254
1.018399480808855
1.024582639672815
1.0228138714606292
1.0233191283108982
1.023174740606874
1.0232159976098358
0.0007608964256584509
1.2413649296698477
0.9651139228153773
1.0401223123424954
1.0183995098187042
1.0245826626179084
1.0228138964866924
1.02331915264479
1.0231747651666014
1.023216022097032
0.0007608959900781492
1.2413649952536405
0.96511

C:\Users\holyb_tkuu01l\AppData\Local\Temp\ipykernel_13144\1919138055.py:6: RuntimeWarning: invalid value encountered in scalar power
  Overlap_matrix[p,q] = (np.pi/(basis[p] + basis[q]))**(3/2)
C:\Users\holyb_tkuu01l\AppData\Local\Temp\ipykernel_13144\1919138055.py:15: RuntimeWarning: invalid value encountered in scalar power
  Kintetic_matrix[p,q] = 3*basis[p]*basis[q]*np.pi**(3/2)/(basis[p] + basis[q])**(5/2)
C:\Users\holyb_tkuu01l\AppData\Local\Temp\ipykernel_13144\2487480282.py:9: RuntimeWarning: invalid value encountered in scalar power
  Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)
C:\Users\holyb_tkuu01l\AppData\Local\Temp\ipykernel_13144\2487480282.py:19: RuntimeWarning: invalid value encountered in scalar power
  Electron_matrix[p,r,q,s] = 2*np.pi**(5/2)/(basis[p] + basis[q])/(basis[r] + basis[s])/(basis[p] + basis[q] + basis[r] + basis[s])**(1/2)


ValueError: array must not contain infs or NaNs